# 5 Data Segregation

In this section we are to split our dataset into two blocks. The first one is going to be used for traning and stored as training_data.csv. The other will be reserved for testing the model.

## 5.1 Install WandB


In [1]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.0 MB/s 
     |████████████████████████████████| 181 kB 63.1 MB/s 
     |████████████████████████████████| 147 kB 62.0 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=26c956ad0d2179217064e1a312360aba87ee8656b0ba013bd63183a74855edad
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


## 5.2 Loading libraries

In [6]:
import logging
import tempfile
import pandas as pd
import os
import wandb
import tensorflow as tf
from sklearn.model_selection import train_test_split

## 5.3 Login to wandb

In [3]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## 5.4 Data Segregation

In [4]:
# global variables

# ratio used to split train and test data
test_size = 0.20

# seed used to reproduce purposes
seed = 57

# reference column to straitify the data
stratify = "status"

# name of the input artifact
artifact_input_name = "phishing-detection-2/preprocessed_data.csv:latest"

# type of the artifact
artifact_type = "data"

In [5]:
# configure logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%Y-%m-%d %H:%M:%S')

# reference for a logging obj
logger = logging.getLogger()

# initiate wandb project
run = wandb.init(project="phishing-detection-2", job_type="split_data")

logger.info("Downloading and reading artifact")
artifact = run.use_artifact(artifact_input_name)
artifact_path = artifact.file()
df = pd.read_csv(artifact_path)

wandb: Currently logged in as: lupamedeiros. Use `wandb login --relogin` to force relogin


2022-07-15 17:43:51 Downloading and reading artifact


In [7]:
# Split firstily in train/test, then we further divide the dataset to train and validation
logger.info("Splitting data into train and test")
splits = {}
splits["train"], splits["test"] = train_test_split(df,
                                                   test_size=test_size,
                                                   random_state=seed,
                                                   stratify=df[stratify])

# Save the artifacts. We use a temporary directory so we do not leave any trace behind
with tempfile.TemporaryDirectory() as tmp_dir:
    
    for split, df in splits.items():

        # MAke the artifact name from the name of the split plus the provided root
        artifact_name = f"{split}.csv"

        # Get the path on disk within the tempo directory
        temp_path = os.path.join(tmp_dir, artifact_name)

        logger.info(f"Uploading the {split} dataset to {artifact_name}")

        # Save then upload to W&B
        df.to_csv(temp_path,index=False)

        artifact = wandb.Artifact(name=artifact_name,
                                  type=artifact_type,
                                  description=f"{split} split of dataset {artifact_input_name}")
        artifact.add_file(temp_path)

        logger.info("Logging artifact")
        run.log_artifact(artifact)

        # This waits for the artifact to be upload to W&B. If you do not add
        # this, te temp directory might be removed before W&B had a chance to 
        # upload the datasets, and upload might fail
        artifact.wait()

2022-07-15 17:58:06 Splitting data into train and test
2022-07-15 17:58:06 Uploading the train dataset to train.csv
2022-07-15 17:58:07 Logging artifact
2022-07-15 17:58:09 Uploading the test dataset to test.csv
2022-07-15 17:58:09 Logging artifact


In [8]:
# close the run
run.finish()